In [1]:
from ultralytics import YOLO
# pip install ensemble-boxes
from ensemble_boxes import weighted_boxes_fusion

In [2]:
pt_model1 = YOLO("runs/detect/yolov9e 0.995 0.825/weights/epoch62.pt")  # load a pretrained model (recommended for training)
pt_model2 = YOLO("runs/detect/yolov9e 0.995 0.825/weights/epoch65.pt")  # load a pretrained model (recommended for training)
# pt_model2 = YOLO("runs/detect/yolov9e 0.995 0.825/weights/epoch65.engine")  # load a pretrained model (recommended for training)

In [3]:
pt_model2.export(format="engine", imgsz=1600, simplify=True, workspace=8, batch=1)

WARNING  TensorRT requires GPU export, automatically assigning device=0
Ultralytics YOLOv8.2.18  Python-3.9.13 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
YOLOv9e summary (fused): 687 layers, 57377171 parameters, 0 gradients, 189.1 GFLOPs



KeyboardInterrupt



In [8]:
from PIL import Image, ImageDraw
import numpy as np
im1 = Image.open('../../../data/images/image_5095.jpg')
im2 = Image.open('../../../data/images/image_5100.jpg')

In [10]:
from PIL import Image, ImageDraw
import numpy as np
im1 = Image.open('2.jpg')
im2 = Image.open('3.jpg')

In [4]:
result1 = pt_model1.predict([im1, im2], imgsz=1600, conf=0.3, iou=0.1, max_det=10)  # max F1, try augment=True and iou
result1_n = [(r.boxes.xyxyn.tolist(), r.boxes.conf.tolist()) for r in result1]
result1_n = [tuple(zip(*r)) for r in result1_n]
result1 = [(r.boxes.xyxy.tolist(), r.boxes.conf.tolist()) for r in result1]
result1 = [tuple(zip(*r)) for r in result1]  # list of tuple[box, conf] in each image in xyxy format

C:\Program Files\Python39\lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


0: 928x1600 7 targets, 124.4ms
1: 928x1600 7 targets, 124.4ms
Speed: 7.6ms preprocess, 124.4ms inference, 932.8ms postprocess per image at shape (1, 3, 928, 1600)


In [5]:
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction

In [6]:
model = AutoDetectionModel.from_pretrained(
    model_type="yolov8",
    model_path='../yolov9e_0.995_0.823_epoch65.pt',
    confidence_threshold=0.1,
    device="cuda",  # or 'cuda:0'
)

06/02/2024 21:14:34 - INFO - albumentations.check_version -   A new version of Albumentations is available: 1.4.8 (you have 1.4.7). Upgrade using: pip install --upgrade albumentations


In [26]:
result = get_sliced_prediction(im1, model, perform_standard_pred=True, postprocess_class_agnostic=True).object_prediction_list
result_n = [([r.bbox.minx / 1520, r.bbox.miny / 870, r.bbox.maxx / 1520, r.bbox.maxy / 870], r.score.value) for r in result]
result = [([r.bbox.minx, r.bbox.miny, r.bbox.maxx, r.bbox.maxy], r.score.value) for r in result]

Performing prediction on 6 slices.


In [11]:
result2 = pt_model2.predict([im1, im2], imgsz=1600, conf=0.3, iou=0.1, max_det=10)  # max F1, try augment=True and iou
result2_n = [(r.boxes.xyxyn.tolist(), r.boxes.conf.tolist()) for r in result2]
result2_n = [tuple(zip(*r)) for r in result2_n]
result2 = [(r.boxes.xyxy.tolist(), r.boxes.conf.tolist()) for r in result2]
result2 = [tuple(zip(*r)) for r in result2]  # list of tuple[box, conf] in each image in xyxy format


0: 928x1600 4 targets, 58.5ms
1: 928x1600 3 targets, 58.5ms
Speed: 8.0ms preprocess, 58.5ms inference, 2.0ms postprocess per image at shape (1, 3, 928, 1600)


In [27]:
result2 = pt_model2.predict([im1], imgsz=(1536, 896), conf=0.3, iou=0.1, max_det=10)  # max F1, try augment=True and iou
result2_n = [(r.boxes.xyxyn.tolist(), r.boxes.conf.tolist()) for r in result2]
result2_n = [tuple(zip(*r)) for r in result2_n]
result2 = [(r.boxes.xyxy.tolist(), r.boxes.conf.tolist()) for r in result2]
result2 = [tuple(zip(*r)) for r in result2]  # list of tuple[box, conf] in each image in xyxy format


0: 1536x896 7 targets, 45.0ms
Speed: 8.0ms preprocess, 45.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1536, 896)


In [7]:
boxes_list = [[r[0] for r in result1_n[0]], [r[0] for r in result2_n[0]]]
scores_list = [[r[1] for r in result1_n[0]], [r[1] for r in result2_n[0]]]
labels_list = [[0] * len(result1_n[0]), [0] * len(result2_n[0])]
weights = [0.1, 1]
boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=0.5, skip_box_thr=1e-4)
boxes = boxes.tolist()
# normalize
boxes = [[x1*1520, y1*870, x2*1520, y2*870] for x1, y1, x2, y2 in boxes]
scores = scores.tolist()
wbf_result = [list(zip(boxes, scores))]

In [13]:
# plot
for im, boxes in zip([im1, im2], result2):
    im = im.copy()
    draw = ImageDraw.Draw(im)
    for (x1, y1, x2, y2), conf in boxes:
        # print(x1, y1, x2, y2, conf)
        print([x1, y1, x2 - x1, y2 - y1], conf)
        draw.rectangle(xy=((x1, y1), (x2, y2)), outline='red')
        draw.text((x1, y1), text=f'{conf:.2f}', fill='red')
    im.show()

[805.4537963867188, 266.6635437011719, 65.88726806640625, 51.044525146484375] 0.8840249180793762
[780.4070434570312, 344.8293762207031, 60.640869140625, 30.552337646484375] 0.8493534326553345
[247.44850158691406, 504.761474609375, 110.45524597167969, 38.27239990234375] 0.8326224684715271
[1415.942138671875, 158.29652404785156, 104.057861328125, 83.5587158203125] 0.797531008720398
[926.0911865234375, 295.0468444824219, 114.6827392578125, 57.679962158203125] 0.8768801093101501
[1131.968994140625, 235.7434844970703, 57.12158203125, 48.26167297363281] 0.8663981556892395
[859.451904296875, 518.83251953125, 67.2882080078125, 43.86517333984375] 0.8398226499557495
